## for TA: Instructions on how to import packages

- `pip install ta`
- `pip install pytz`
- `pip install ib_insync`
- `pip install asyncio`

In [ ]:
# Imports
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.scanner import ScannerSubscription
from ibapi.contract import Contract
from ibapi.order import *
import ta
import numpy as np
import pandas as pd
import pytz 
import math
from datetime import datetime, timedelta, timezone
import threading
import time
import random
import math
import asyncio

#testing
import ib_insync
from ib_insync import *
from collections import deque

## Stock Scanner
- NOTE: this file should only be ran during market hours - there is no way to run this using historical data
- File selects top 5 stocks (currently hardcoded) based off of top percent gain metric 
- File then shows user the current top 5 stocks, with a refresh of the top percent gain metric happening in 30 second intervals
- User will be shown only the top 5 stocks at a time, and can select "n" if they are not satisfied with the picked stocks. Program will automatically run with updated stocks every 30 seconds and will terminate when user types in "y", indicating they like the stocks displayed and want to trade those stocks
- File will then locally save the user's selected stocks

# Step 1: Stock Scanner

In [ ]:
class TradeApp(EWrapper, EClient):
    
    def __init__(self):
        EClient.__init__(self,self)
        self.stockStack = deque()
    
    def scannerData(self, reqId, rank, contractDetails, distance, benchmark, projection, legsStr):
        super().scannerData(reqId, rank, contractDetails, distance, benchmark, projection, legsStr)
        getSymbol = contractDetails.contract.symbol
        self.stockStack.append(getSymbol)

In [ ]:
def usStkScan(asset_type="STK",asset_loc="STK.US.MAJOR",scan_code = "TOP_PERC_GAIN"):
    scanSub = ScannerSubscription()
    scanSub.numberOfRows = 5
    scanSub.abovePrice = 20
    scanSub.belowPrice = 80
    scanSub.aboveVolume = 1000000
    scanSub.instrument = asset_type
    scanSub.locationType = asset_loc
    scanSub.scanCode = scan_code
    return scanSub

In [ ]:
def websocket_con():
    app.run()

In [ ]:
app = TradeApp()
app.connect(host = '127.0.0.1', port = 7497, clientId = 23)
con_thread = threading.Thread(target=websocket_con)
con_thread.start()
time.sleep(1)

In [ ]:
app.reqScannerSubscription(1,usStkScan(),[],[])
time.sleep(30)

In [ ]:
traderLike = False
pickedStocks = None

while not traderLike:
    
    app.reqScannerSubscription(1,usStkScan(),[],[]) #append to stack regardless
    
    viewStocks = app.stockStack #view current stack
    currStocks = []
    
    for i in range(5):
        ret = viewStocks.pop()
        currStocks.append(ret)
    
    time.sleep(30)
    print("5 stocks: ",currStocks)
    val = input("Do you want to trade these 5 stocks? y/n")
    
    if val == "y":
        traderLike = True
        pickedStocks = currStocks
        app.disconnect()
        print("Congrats! Your picked stocks were: ",pickedStocks)
        print("Happy Trading :)")
        break
    
    currStocks = [] #reinitialize list to hold the top of stack 

In [ ]:
app.disconnect()

In [ ]:
pickedStocks

# Step 2: Algo Trading Bot

In [ ]:
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        
        #used for caching historical data for placing dummy trades outside of market hours
        self.data = [] 
        
    #historical backtest data
    def historicalData(self, reqId, bar):
        #cache historical data
        self.data.append([bar.date, bar.high, bar.low, bar.close])
        bot.on_bar_update(reqId,bar,False)
             
    #on historical data end
    def historicalDataEnd(self, reqId, start, end):
        print(reqId)
    
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextValidOrderId = orderId
              
    def error(self, id, errorCode, errorMsg):
        print("TWS Error Code: ", errorCode)
        print("Message for Error Code: ",errorMsg)
    
    # On Realtime Bar after historical data finishes
    def historicalDataUpdate(self, reqId, bar):
        try:
            bot.on_bar_update(reqId,bar,True)
        except Exception as e:
            print(e)

    # Listen for realtime bars
    def realtimeBar(self, reqId, time, open_, high, low, close,volume, wap, count):
        print("Real-time bars during trading hours")
        super().realtimeBar(reqId, time, open_, high, low, close, volume, wap, count)
        try:
            realTimeBar = Bar(open_, low, high, close, volume, time)
            bot.on_bar_update(reqId, realTimeBar,True)
        except Exception as e:
            print(e)


In [ ]:
#Bar Object
class Bar:
    open = 0
    low = 0
    high = 0
    close = 0
    volume = 0
    date = datetime.now()
    def __init__(self, open_=None, low=None, high=None, close=None, volume=None, time=None):
        self.open = open_ or 0
        self.low = low or 0
        self.high = high or 0
        self.close = close or 0
        self.volume = volume or 0
        self.date = time or datetime.now()

In [ ]:
# Bot Logic
class Bot:
    ib = None
    bars = []
    reqId = 1
    global orderId
    smaPeriod = 15
    symbol = ''
    currentBar = Bar() #initialize first bar to be all 0's
    initialbartime = datetime.now().astimezone(pytz.timezone("America/New_York"))
    bracketSubmit = False

    def __init__(self, stockTicker):
        
        #connect to IB on init
        self.ib = IBApi()
        
        #use a random port Id everytime to ensure little chance of getting duplicate orderIds
        randPortNum = np.random.randint(50, 250)
        self.ib.connect('127.0.0.1', 7497, randPortNum)
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        ib_thread.start()
        time.sleep(1)

        #grab current stockTicker that scanner gives us
        self.symbol=stockTicker
        self.barsize = 1
        
        minQuantity = ' min'
        if (int(self.barsize) > 1):
            minQuantity = ' mins'
        
        #create contract object
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        
        self.ib.reqIds(-1) 
        
        #request market data
        print("Requesting market data...")
        self.ib.reqHistoricalData(self.reqId, contract, '', '2 D', str(self.barsize)+minQuantity, 'TRADES',1,1,True,[])
        
        '''
        TESTING FOR REALTIME DATA
        '''
        # implicitly calls on_bar_update
        self.ib.reqRealTimeBars(self.reqId, contract, 5, "MIDPOINT", True, [])

    
    #listen to socket in seperate thread
    def run_loop(self):
        self.ib.run()
    
    #bracket order
    def bracketOrder(self, parentOrderId, action, quantity, profitTarget, stopLoss):
        print("within bracket order, parentOrderId is: ",parentOrderId)
        
        #initial entry
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        
        #create parent order 
        parent = Order()
        parent.orderId = parentOrderId  # increment local copy of TWS global orderId, was parentOrderId+1
        parent.orderType = 'MKT'
        parent.action = action
        parent.totalQuantity = quantity
        parent.transmit = False 
        
        print("within bracket order, parent order made, parentOrderId is: ",parent.orderId)
        
        # make a target order
        profitOrder = Order()
        profitOrder.orderId = parentOrderId+1 # increment local copy of TWS global orderId
        profitOrder.orderType = 'LMT'
        profitOrder.action = "SELL" if action == "BUY" else "BUY" 
        profitOrder.totalQuantity = quantity
        profitOrder.parentId = parentOrderId 
        profitOrder.lmtPrice = round(profitTarget,3)
        profitOrder.transmit = False 
    
        print("within bracket order, target order made, profitOrder.orderId is: ",profitOrder.orderId)
        
        # make a loss order
        stopOrder = Order()
        stopOrder.orderId = parentOrderId+2 # increment local copy of TWS global orderId
        stopOrder.orderType = 'STP'
        stopOrder.action = "SELL" if action == "BUY" else "BUY" 
        stopOrder.totalQuantity = quantity
        stopOrder.parentId = parentOrderId 
        stopOrder.auxPrice = round(stopLoss,3)
        stopOrder.transmit = True 
        
        print("within bracket order, loss order made, stopOrder.orderId is: ",stopOrder.orderId)
        return [parent, profitOrder, stopOrder]
   
    #Pass realtime bar data back to our bot object
    def on_bar_update(self, reqId, bar,realtime):
        global orderId
        
        # helper function to truncate decimal precision on prices
        def round_nearest2(x, a):
            return round(round(x / a) * a, -int(math.floor(math.log10(a))))
        
        #Historical Data to catch up
        if (realtime == False):
            self.bars.append(bar)
       
        else:

            bartime = datetime.strptime(bar.date,"%Y%m%d %H:%M:%S").astimezone(pytz.timezone("America/New_York"))
            minutes_diff = (bartime-self.initialbartime).total_seconds() / 60.0 
            self.currentBar.date = bartime
            
            lastBar = self.bars[len(self.bars)-1]
            getBarsizeInt = int(self.barsize)
            
            #On Bar Close
            print("minutes_diff: ",minutes_diff)
            print(getBarsizeInt)
            if (minutes_diff > 0 and math.floor(minutes_diff) % getBarsizeInt == 0):
               
                self.initialbartime = bartime 
                
                #Entry - If we have a higher high, a higher low and we cross the 50 SMA Buy
                #1.) SMA
                closes = []
                for bar in self.bars:
                    closes.append(bar.close)
                self.close_array = pd.Series(np.asarray(closes))
                
                self.sma = ta.trend.sma_indicator(self.close_array,self.smaPeriod,True) #original
                
                # new addition: sma for 10, sma for 20
                self.sma10 = ta.trend.sma_indicator(self.close_array,10,True)
                self.sma20 = ta.trend.sma_indicator(self.close_array,20,True)   
                print("SMA : " + str(self.sma[len(self.sma)-1]))
                
                #2.) Calculate Higher Highs and Lows
                lastLow = self.bars[len(self.bars)-1].low
                lastHigh = self.bars[len(self.bars)-1].high
                lastClose = self.bars[len(self.bars)-1].close
                
                print("bar.close: ",bar.close)
                print("self.sma10[len(self.sma10)-1] is: ",self.sma10[len(self.sma10)-1])
                print("self.sma20[len(self.sma20)-1] is: ",self.sma20[len(self.sma20)-1])

                # Check Criteria
                if (bar.close > lastHigh
                    and self.currentBar.low > lastLow
                    and bar.close > str(self.sma[len(self.sma)-1])
                    and lastClose < str(self.sma[len(self.sma)-2])):
        
                    print("made it within criteria!")
                    
                    # Bracket Order 3% Profit Target 1% Stop Loss
                    # used to truncate price to less decimal precion - fixes Bug-10
                    profitTarget = round_nearest2(bar.close*1.03, 0.05)
                    stopLoss = round_nearest2(bar.close*0.99, 0.05)
                    print("profitTarget is: ",profitTarget)
                    print("stopLoss is: ",stopLoss)
                    quantity = 4000
                    
                    # get next valid global TWS order id, make it the new parent id in the bracket order
                    order_id = self.ib.nextValidOrderId
                    bracket = self.bracketOrder(order_id,"BUY",quantity, profitTarget, stopLoss)
                    
                    # increment global TWS orderId 3 times 
                    self.ib.nextValidOrderId
                    self.ib.nextValidOrderId
                    self.ib.nextValidOrderId
                    
                    contract = Contract()
                    contract.symbol = self.symbol.upper()
                    contract.secType = "STK"
                    contract.exchange = "SMART"
                    contract.currency = "USD"
                    
                    #Place Bracket Order
                    for o in bracket:
                        print("Order action: ",o.action)
                        o.ocaType = 1 #new addition, remove if it breaks 
                        time.sleep(5)
                        self.ib.placeOrder(o.orderId,contract,o) 
                        time.sleep(5)
                        self.bracketSubmit = True
                    
                    self.ib.nextValidOrderId
                    self.ib.nextValidOrderId
                    self.ib.nextValidOrderId
                    print("bracketSubmit value is: ",self.bracketSubmit)
                    
                    # testing for actionaItem-1
                    # disconnet from port upon submitting bracket order
                    if self.bracketSubmit:
                        print("Order submitted, disconnecting from TWS API now...")
                        self.ib.disconnect()
                                            
                #Bar closed append
                self.currentBar.close = bar.close
                print("New bar!")
                self.bars.append(self.currentBar)
                self.currentBar = Bar()
                self.currentBar.open = bar.open
       
        #Build  realtime bar
        if (self.currentBar.open == 0):
            self.currentBar.open = bar.open
        if (self.currentBar.high == 0 or bar.high > self.currentBar.high):
            self.currentBar.high = bar.high
        if (self.currentBar.low == 0 or bar.low < self.currentBar.low):
            self.currentBar.low = bar.low


# Step 3: Create Bracket Order for each stock

In [ ]:
# create a bracket order for each of our winning stocks
# original code, bring back if fixes for Bug-11 fail
for stock in pickedStocks:
    bot = Bot(stock)
    time.sleep(20)